In [22]:
import pandas as pd
import openpyxl
import numpy as np

In [ ]:
pos = pd.read_excel("C:/Users/evasquez/OneDrive - Bolsa de Cereales de Buenos Aires/Documentos - Instituto Estudios Económicos/Pedidos/2024/Diferencias_FOB/Data/base_precios_bolsa.xlsx")
pos["Fecha"] = pd.to_datetime(pos["Fecha"], format="%m/%d/%Y")
pos["Fecha"] = pos["Fecha"].dt.strftime("%Y-%m-%d")

#ver la estructura de datos de la tabla
print(pos.dtypes)

In [ ]:
# Convertir la columna "Año" de float64 a int64
pos["Año"] = pos["Año"].astype("Int64")
pos["Mes"] = pos["Mes"].astype("Int64")
print(pos.dtypes)

In [5]:
pos = pos.dropna(subset=["Año", "Mes"])
pos["posicion"] = pd.to_datetime(pos["Año"].astype(str) + '-' + pos["Mes"].astype(str) + '-01')


In [6]:
#Crear una columna con las diferencia de meses 
pos['Fecha'] = pd.to_datetime(pos['Fecha'])
pos['posicion'] = pd.to_datetime(pos['posicion'])

pos['time_delta_months'] = pos['posicion'].dt.to_period('M').view(dtype='int64') - pos['Fecha'].dt.to_period('M').view(dtype='int64')

pos['Embarque'] =  't+' + pos['time_delta_months'].astype(str)

In [7]:
# Filtrar Fecha desde 2022 en adelante
pos = pos[pos['Fecha'] >= '2020-01-01']
pos["Producto"].replace("Trigo 11.5%", "Trigo", inplace=True)

In [8]:
colors = pd.read_excel("C:/Users/evasquez/OneDrive - Bolsa de Cereales de Buenos Aires/Documentos - Instituto Estudios Económicos/Pedidos/2024/Diferencias_FOB/Reunion_Ministerio_1508/análisis/Base_FullSites.xlsx")

In [ ]:
colors["Fecha"] = pd.to_datetime(colors["Fecha"], format="%m/%d/%Y")
colors["Fecha"] = colors["Fecha"].dt.strftime("%Y-%m-%d")

#filtro fecha
colors = colors[colors['Fecha'] >= '2020-01-01']
#ver la estructura de datos de la tabla
print(colors.dtypes)

In [10]:
pos['Fecha'] = pd.to_datetime(pos['Fecha'])
colors['Fecha'] = pd.to_datetime(colors['Fecha'])
df = pd.merge(pos, colors, on=["Fecha", "Producto", "Mes", "Año"], how="left")

In [11]:
#exportar a excel
df.to_excel("C:/Users/evasquez/OneDrive - Bolsa de Cereales de Buenos Aires/Documentos - Instituto Estudios Económicos/Pedidos/2024/Diferencias_FOB/Reunion_Ministerio_1508/análisis/base_analisis.xlsx", index=False)
del pos

In [ ]:
#LIMPIO
df['Mes_Fecha'] = df['Fecha'].dt.month

resumen = df.groupby(['Fecha', 'Producto', 'Embarque'])['Empresa'].count().reset_index()
resumen.columns = ['Fecha', 'Producto', 'Embarque', 'Cantidad_Empresa']

df_junto = pd.merge(df, resumen, on=['Fecha', 'Producto', 'Embarque'], how='left')

# Me indica la cantidad de empresas que reportaron precios por mes, producto y embarque
reporte_qempresa = df_junto.groupby(['Mes_Fecha', 'Producto', 'Embarque'])['Cantidad_Empresa'].mean().reset_index().round(2)
print(reporte_qempresa)

In [ ]:
# Frecuencia de empresa que informa por producto
df_frecuencia = df[df["Producto"] == "Soja"]
frecuencia_empresa = df_frecuencia['Empresa'].value_counts()
print(frecuencia_empresa)

In [ ]:
# Producto que más se recibe información
frecuencia_producto = df['Producto'].value_counts()
print(frecuencia_producto)
df_filtered = df[(df['Fecha'].dt.year >= 2023)]
cantidad_registros = df_filtered['Producto'].value_counts()
print(cantidad_registros)

In [ ]:
colors['Mes_Fecha'] = colors['Fecha'].dt.month
print(colors.dtypes)

In [16]:
colors.dropna(subset=['Valor'], inplace=True)

colors["posicion"] = pd.to_datetime(colors["Año"].astype(str) + '-' + colors["Mes"].astype(str) + '-01')
colors['posicion'] = pd.to_datetime(colors['posicion'])

colors['time_delta_months'] = colors['posicion'].dt.to_period('M').view(dtype='int64') - colors['Fecha'].dt.to_period('M').view(dtype='int64')

colors['Embarque'] ='t+' + colors['time_delta_months'].astype(str)


In [ ]:
# frecuencia de embarques por producto
color_freq = colors[colors['Producto'] == 'Trigo'].groupby('Embarque')['Valor'].value_counts()
color_freq = color_freq.loc[:, 'Naranja']  ## RECUERDA CAMBIAR EL COLOR
print(color_freq)

In [ ]:
# frecuencia de Colores por producto
frecuencia_valor_trigo = colors[colors['Producto'] == 'Soja']['Valor'].value_counts()
print(frecuencia_valor_trigo)

In [ ]:
#color por mes de embarque
frecuencia_valor_verde = colors[(colors['Producto'] == 'Aceite de Girasol') & (colors['Valor'] == 'Rojo')]['Mes'].value_counts()
frecuencia_valor_verde = frecuencia_valor_verde.sort_index()
print(frecuencia_valor_verde)

In [ ]:
#color por t
soja_colors = colors[colors['Producto'] == 'Aceite de Girasol']
frecuencia_embarque_valor = soja_colors.groupby(['Embarque', 'Valor']).size().reset_index(name='Frecuencia')
frecuencia_embarque_valor
frecuencia_embarque_valor.pivot(index='Embarque', columns='Valor', values='Frecuencia')

In [ ]:
# Registros por producto y año de la fecha
df_count = df.groupby(['Producto', 'Año_Fecha']).size().reset_index(name='Cantidad_Registros')
print(df_count)


In [ ]:
#
df_broker_trigo = df[(df['Categoria'] == 'empresa') & (df['Producto'] == 'Soja')]
registro_agrupado = df_broker_trigo.groupby(['Año_Fecha', 'Empresa']).size().reset_index(name='Cantidad_Registros')
print(registro_agrupado)

PROMEDIOS

In [23]:
df_promedio = df
df_promedio['Año_Fecha'] = df_promedio['Fecha'].dt.year
colors_promedio = colors
colors_promedio['Año_Fecha'] = colors_promedio['Fecha'].dt.year


In [ ]:
empresa_freq = df_promedio[df_promedio['Producto'] == 'Soja'].groupby(['Año_Fecha', 'Empresa']).size().reset_index(name='Frecuencia')
empresa_freq_pivot = empresa_freq.pivot(index='Año_Fecha', columns='Empresa', values='Frecuencia')
print(empresa_freq_pivot)

In [41]:
#
productos_interes = ['Soja', 'Trigo', 'Aceite de Girasol']
df_filtrado = colors[colors['Producto'].isin(productos_interes)]

# Agrupar por 'Mes_Fecha' y contar las frecuencias de los valores para cada producto
df_productos_freq = df_filtrado.groupby(['Producto', 'Mes_Fecha'])['Valor'].value_counts()

# Convertir la serie en un DataFrame
df_productos_freq = df_productos_freq.to_frame(name='Frecuencia').reset_index()


In [ ]:
df_broker_trigo = df[df['Categoria'] == 'broker']
broker_count_by_month = df_broker_trigo[df_broker_trigo['Producto'] == 'Trigo'].groupby('Mes')['Empresa'].count()
print(broker_count_by_month)